In [52]:
import sys
sys.version_info

# pip install kinopoisk-api-unofficial-client

from kinopoisk_unofficial.kinopoisk_api_client import KinopoiskApiClient
from kinopoisk_unofficial.request.films.film_request import FilmRequest
import time
import pickle
from kinopoisk_unofficial.request.staff.staff_request import StaffRequest
import re
from py2neo import Graph

In [2]:
api_client = KinopoiskApiClient("3234199f-eb6e-4667-ae9f-9a26a19feff0")

In [5]:
responses = {}
i=200
while True:
    # time.sleep(1)
    try :
        request = FilmRequest(i)
        response = api_client.films.send_film_request(request)
        responses[i] = (response.__dict__)
        print("film with id", i, "added")
    except:
        print("No id", i)

    if len(responses) >= 200:
        break
    i+=1
    

No id 200
No id 201
No id 202
No id 203
No id 204
No id 205
No id 206
No id 207
No id 208
No id 209
No id 210
No id 211
No id 212
No id 213
No id 214
No id 215
No id 216
No id 217
No id 218
No id 219
No id 220
No id 221
No id 222
No id 223
No id 224
No id 225
No id 226
No id 227
No id 228
No id 229
No id 230
No id 231
No id 232
No id 233
No id 234
No id 235
No id 236
No id 237
No id 238
No id 239
No id 240
No id 241
No id 242
No id 243
No id 244
No id 245
No id 246
No id 247
No id 248
No id 249
No id 250
No id 251
No id 252
No id 253
No id 254
No id 255
No id 256
No id 257
No id 258
No id 259
No id 260
No id 261
No id 262
No id 263
No id 264
No id 265
No id 266
No id 267
No id 268
No id 269
No id 270
No id 271
No id 272
No id 273
No id 274
No id 275
No id 276
No id 277
No id 278
No id 279
No id 280
No id 281
No id 282
No id 283
No id 284
No id 285
No id 286
No id 287
No id 288
No id 289
No id 290
No id 291
No id 292
No id 293
No id 294
No id 295
No id 296
No id 297
film with id 298 add

In [12]:
with open('films.pickle', 'wb') as handle:
    pickle.dump(responses, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
res == responses

True

In [21]:
film_staff = {}
for film in res.keys():
    request = StaffRequest(film)
    response = api_client.staff.send_staff_request(request)
    film_staff[film] = response

In [23]:
with open('film_staff.pickle', 'wb') as handle:
    pickle.dump(film_staff, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [25]:
staff == film_staff

True

In [2]:
with open('films.pickle', 'rb') as handle:
    res = pickle.load(handle)

In [3]:
with open('film_staff.pickle', 'rb') as handle:
    staff = pickle.load(handle)

In [163]:
input_text = ""

new_film_names = {}
for film in res.keys():
    try :
        film_name = re.sub(r"\W+", "", res[film]["film"].__dict__["name_original"].replace(" ", ""))
    except:
        try:
            film_name = re.sub(r"\W+", "", res[film]["film"].__dict__["name_en"].replace(" ", ""))
        except:
            film_name = "m" + str(film)

    try:
        flag = int(film_name[0])
        film_name = "m" + film_name
    except:
        pass
    new_film_names[film] = film_name
    input_text += f'CREATE ({film_name}:Movie '
    input_text += "{"
    tagline = str(res[film]["film"].__dict__["slogan"]).replace("'", "").replace('"', '')
    input_text += f'title:"{res[film]["film"].__dict__["name_ru"]}", released:{res[film]["film"].__dict__["year"]}, tagline:"{tagline}"'
    input_text += "}) "



people_id = []
people_new_name = {}
for film in staff.keys():
    for person in staff[film].__dict__["items"]:
        if person.__dict__["profession_text"] in ["Режиссеры", "Актеры", "Продюсеры", "Сценаристы"] and person.__dict__["staff_id"] not in people_id:
            people_id.append(person.__dict__["staff_id"])

            if person.__dict__["name_en"] == "":
                person_name = "p" + str(person.__dict__["staff_id"])
            else:
                person_name = re.sub(r"\W+", "", person.__dict__["name_en"].replace(" ", ""))
            
            if person_name in people_new_name.values():
                person_name += str(person.__dict__["staff_id"])
            people_new_name[person.__dict__["staff_id"]] = person_name
            input_text += f'CREATE ({person_name}:Person '
            input_text += "{"
            input_text += f'name:"{person.__dict__["name_ru"]}"'
            input_text += "}) "


roles = {"Режиссеры":"DIRECTED", "Продюсеры":"PRODUCED", "Сценаристы":"WROTE"}

for film in staff.keys():
    for person in staff[film].__dict__["items"]:
        if person.__dict__["profession_text"] in roles.keys():
            input_text += f'CREATE ({people_new_name[person.__dict__["staff_id"]]})-[:{roles[person.__dict__["profession_text"]]}]->({new_film_names[film]}) '
        elif person.__dict__["profession_text"] == "Актеры":
            input_text += f'CREATE ({people_new_name[person.__dict__["staff_id"]]})-[:ACTED_IN '
            input_text += "{"
            input_text += f'roles:["{person.__dict__["description"]}"]'
            input_text += "}"
            input_text += f']->({new_film_names[film]}) '



In [164]:
final = open("final.txt", "w", encoding = "utf-8")
final.write(input_text)
final.close()

In [165]:
graph = Graph("bolt://localhost:7687", auth=("GGG", "123456789"))
graph.run(input_text)

   ---------------------------------------- 0.0/177.2 kB ? eta -:--:--
   ------------------------- -------------- 112.6/177.2 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 177.2/177.2 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
